# HelioCloud Storage + Burst + SDO

* HelioCloud is an AWS Cloud environment that is user-friendly
* Daskhub is a cloud Notebooks setup that allows parallel processing via Dask
* Dask lets you temporarily throw lots of CPUs at a problem
* S3 is the big cheap AWS storage
* SDO is a mission with lots of image data

Here we combine a few HelioCloud demos with our big data fileRegistry to tackle 1 year of SDO data.  The data is in AWS S3 and we do not copy the files over, but instead have CPUs at AWS access it directly

* 1 year of SDO 94A EUV images from AIA is 129,758 files, each 14MB, totalling 1.8 TB.
* This code calculates a simple irradiance
* If done serially on your laptop, it would take 27 hours
* HelioCloud takes 25 minutes (1467 sec) to analyze through 1 year of SDO data

(More fun stats: that's 88 files/second, also 1 GB/sec to do the full analysis (which is 2x the read speed of a SATA SSD.  It is 2x faster than it would take to just copy the files off a local disk and 8x faster than copying via gigabit internet).


In [ ]:
import cloudcatalog
import boto3
import dask
import io
import logging
import time
import re
import pickle
import astropy.io.fits
from dask.distributed import Client
from dask_gateway import Gateway, GatewayCluster

# HelioCloud shared cloud file registry (cloudcatalog)

This is a simple standard for any dataset that enables users to access it via an API or directly.  The short definition is:
    * S3 disks have a 'catalog.json' describing their datasets
    * Each dataset has a <dataid>_YYYY.csv index file of its contents
    * These indexes have the form "time, s3_location, filesize" (plus optional metadata)
    
Let's walk through this.

# List of all known 'catalogs'

In [ ]:
# this is pre-release so the main registry is on a temporary site
cr=cloudcatalog.CatalogRegistry()

# Let us see which clouds are known to the HelioCloud network
cat = cr.get_catalog()
print("List of clouds via get_catalog",cat,'\n')
reg = cr.get_registry()
print("Or just the S3 endpoints via get_registry:",reg,'\n')
link = cr.get_entries()
print("get_entries:",link,'\n')

# Now let us grab the specific cloud containing SDO
url = cr.get_endpoint("HelioCloud, including SDO")
print("SDO result for get_endpoint:",url)

## For a given S3 location, print the first in the list of all available datasets

If you want to print the entire list, change the variable "show_all" to True. Be warned that the list will be extremely long and difficult to parse. Remember that you can right click on the cell and select "Clear Cell Output"

In [ ]:
show_all = False # True

fr=cloudcatalog.CloudCatalog("s3://gov-nasa-hdrl-data1/")

if show_all:
    print("All datasets at this HDRL S3:\n",fr.get_entries_dict())
else:
    print("First entry in the list of all datasets at this HDRL S3:\n",fr.get_entries_dict()[0])

## Print metadata associated with AIA 94A

In [ ]:
frID = "aia_0094"
myjson = fr.get_entry(frID)
start, stop = myjson['start'], myjson['stop']
print(frID,start,stop)

## Optionally you can choose a subsetted time range (faster testing)

In [ ]:
subset = True
if subset: start, stop = '2020-01-01T00:00:00Z', '2020-12-31T23:59:59Z'

## Get the entire list of SDO files for that AIA ID

In [ ]:
file_registry1 = fr.request_cloud_catalog(frID, start_date=start, stop_date=stop, overwrite=False)
# And convert that richer data to a list of files to process
filelist = file_registry1['datakey'].to_list()

In [ ]:
file_registry1

# Actual Analysis code

Here's our analysis routines, that operate on a single FITS file and returns the results of our 'work_on_data()' function.

In [ ]:
def DO_SCIENCE(mydata):
    # you can put better science here
    science_result = mydata.mean()
    return science_result

# these are variable helpful handler functions
def s3url_to_bucketkey(s3url: str): # -> Tuple[str, str]:
    """
    Extracts the S3 bucket name and file key from an S3 URL.
    e.g. s3://mybucket/mykeypart1/mykeypart2/fname.fits -> mybucket, mykeypart1/mykeypart2/fname.fits
    """
    name2 = re.sub(r"s3://","",s3url)
    s = name2.split("/",1)
    return s[0], s[1]

def process_fits_s3(s3key:str): # -> Tuple[str, float]:
    """ For a single FITS file, read it from S3, grab the header and
        data, then do the DO_SCIENCE() call of choice
    """
    sess = boto3.session.Session() # do this each open to avoid thread problem 'credential_provider'
    s3c = sess.client("s3")
    mybucket,mykey = s3url_to_bucketkey(s3key)
    try:
        fobj = s3c.get_object(Bucket=mybucket,Key=mykey)
        rawdata = fobj['Body'].read()
        bdata = io.BytesIO(rawdata)
        hdul = astropy.io.fits.open(bdata,memmap=False)        
        date = hdul[1].header['T_OBS']
        science_result = DO_SCIENCE(hdul[1].data)
    except:
        print("Error fetching ",s3key)
        date, science_result = None, None
        
    return date, science_result


# Accessing S3 from anywhere
The power of cloud is moving compute (CPUs) to the data, instead of hauling data over internet.  But if you want to play, here is a serial example of fetching the first 10 SDO files and processing them anywhere.  It is serial and slow-- roughly 66x slower than using parallel processing and 128x or more slow if you are fetching from S3 to your local machine.

In [ ]:
# Serial test it on the first ten files, so slow
now=time.time()
for i in range(10):
    results = process_fits_s3(filelist[i])
    print(results)
print(time.time()-now)

# Setting up the Dask 'burst' Configuration
This is pretty standard daskhub configuration from the HelioCloud dask demos, that works to burst tasks into the cloud.

We default to setting testing as 'True' so it only runs on 10,000 SDO files. If you set it to 'False', it will go through all the files.

In [ ]:
testing = True
if testing:
    s3_files = filelist[0:1000] # small test set to test
else:
    s3_files = filelist
print(len(s3_files))

Here are some Dask parameters for setting up your run.  These are non-optimized values, feel free to play with them to improve performance.

In [ ]:
# number of workers to use, for automatic scaling, our max number
n_workers = 10 # 10-50 works well, more workers actually went slower
# memory per worker (in Gb), typically 1, 2 or 4GB
w_memory = 4
# cores per worker, must be 1-4
w_cores = 4
# use Manual (if False, then uses Automatic scaling)
use_manual_scaling = False

useGUI = False # set true if you want the Dask widget as well

Now we initialzie the Dask gateway and cluster, using your above parameters, to set up the virtual machines that will subsequently operate on the data.

In [ ]:
# from the daskhub tutorial, setting up dask
gateway = Gateway()
options = gateway.cluster_options()
options.worker_cores = w_cores
options.worker_memory = w_memory

In [ ]:
# initialize cluster and create client, takes < 15 seconds

cluster = gateway.new_cluster(options)
client = cluster.get_client() # can also use 'client=Client(cluster)'
cluster.adapt(minimum=1, maximum=n_workers)

# This calls the widget
if useGUI: cluster

In [ ]:
client # let us take a look at it

# Now the actual algorithm work and runtime

In this case, our earlier 'file_registry1' is our set of fully qualified s3:// objects.  This code takes around 5 minutes for 10,000 files and 25 minutes for the full 130K files.

In [ ]:
now=time.time()

# simple version step 1, do it
time_irrad = client.map(process_fits_s3, s3_files)

# optional spot-check that jobs were sent
print(time_irrad[0:4])

# simple version step 2, gather results
all_data = client.gather(time_irrad)

print("Done! Completed in time ",(time.time()-now)/60.0,"minutes, on",len(all_data),"files")

Remember to always shut down your cluster (so you are not spending $ on unused resources.)

In [ ]:
# always shutdown
cluster.shutdown()

# Done, now do something with the results
Time to save your results, plot it, and carry on with your analysis.

In [ ]:
# let us save this
with open('SDO_test-big.pickle','wb') as fout:
    pickle.dump(all_data, fout)

In [ ]:
# Goodness check-- did any files not process?
a = [a for a in all_data if a[0] is None]
print("Bad fields: ",len(a))
plotme = [a for a in all_data if a[0] is not None]
print("Good fields: ",len(plotme))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline 
# Have Matplotlib create vector (svg) instead of raster (png) images
#%config InlineBackend.figure_formats = ['svg'] 
#plt.figure()
plt.plot_date(*zip(*plotme))
plt.xticks(rotation=90)
plt.show()